In [2]:
%matplotlib inline

import numpy as np
import theanolm
import os
import os.path
import heapq
import random
import math
import sys
import re
import shutil

sys.path.insert(0, '/home/ec2-user/kklab/Projects/lrlp/scripts/oov_translate')
from utils import *

In [3]:
### language model directory
tmp_dir = exp_dir+"oov_trans_neural/"
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

In [4]:
### hyper parameters to this method, to be used as parameters of this functions 
cluster_size = 50
net_arch = "lstm1500"
learning_rate = 1.0
optimization_method = "nesterov"

### language_model
language_model = "{}_{}_{}_{}".format(cluster_size, \
                                      net_arch.split("/")[-1], \
                                      learning_rate, \
                                      optimization_method)

lm_in_domain_path = tmp_dir+"lm_"+language_model+"_in_domain"
train_in_domain_tmp = tmp_dir+"train_in_domain.txt"

if not os.path.exists(train_in_domain_tmp):
    shutil.copyfile(train_in_domain, train_in_domain_tmp)

### word clustering
print("cd "+tmp_dir+"; "+\
      wcluster+\
      " --text "+train_in_domain_tmp+\
      " --c "+str(cluster_size))

### adapt the output of wcluster to something that theanoLM recognizes as vocab
vocab_path = tmp_dir+"train_in_domain-c"+str(cluster_size)+"-p1.out/paths"
if not os.path.exists(vocab_path+"3"):
    sh("cut -f1,2 "+vocab_path+" > "+vocab_path+"2")
    sh("awk -F $'\t' ' { t = $1; $1 = $2; $2 = t; print; } ' OFS=$'\t' "+\
       vocab_path+"2"+\
       " > "+vocab_path+"3")

print("theanolm train "+lm_in_domain_path+\
      " --training-set "+train_in_domain+\
      " --validation-file "+dev_in_domain+\
      " --vocabulary "+vocab_path+"3"+\
      " --vocabulary-format classes "+\
      " --architecture "+net_arch+\
      " --learning-rate "+str(learning_rate)+\
      " --optimization-method "+optimization_method+\
      " --stopping-criterion no-improvement"+\
      " --validation-frequency 8"+\
      " --patience 4")

cd /home/ec2-user/kklab/Projects/lrlp/experiment_elisa.il3-eng.y1r1.v2/oov_trans_neural/; /home/ec2-user/kklab/src/brown-cluster/wcluster --text /home/ec2-user/kklab/Projects/lrlp/experiment_elisa.il3-eng.y1r1.v2/oov_trans_neural/elisa.il3-eng.train.y1r1.v2.true.eng.txt --c 50
theanolm train /home/ec2-user/kklab/Projects/lrlp/experiment_elisa.il3-eng.y1r1.v2/oov_trans_neural/lm_50_lstm1500_1.0_nesterov_in_domain --training-set /home/ec2-user/kklab/data/ELISA/evals/y1/JMIST/elisa.il3.package.y1r1.v2/train/elisa.il3-eng.train.y1r1.v2.true.eng --validation-file /home/ec2-user/kklab/data/ELISA/evals/y1/JMIST/elisa.il3.package.y1r1.v2/dev/elisa.il3-eng.dev.y1r1.v2.true.eng --vocabulary /home/ec2-user/kklab/Projects/lrlp/experiment_elisa.il3-eng.y1r1.v2/oov_trans_neural/elisa.il3-eng.train.y1r1.v2.true.eng-c50-p1.out/paths3 --vocabulary-format classes  --architecture lstm1500 --learning-rate 1.0 --optimization-method nesterov --stopping-criterion no-improvement --validation-frequency 8 --pat

In [8]:
train_non_domain = tmp_dir+"train_non_domain"
total_num_in_domain = get_file_length(train_in_domain)
if not os.path.exists(train_non_domain):
    total_num_non_domain = random_sample(train_non_domain_all, \
                                         total_num_in_domain, \
                                         train_non_domain)
else:
    print("Sample file exists at: "+train_non_domain)
    total_num_non_domain = get_file_length(train_non_domain)
print("total_num_in_domain: "+str(total_num_in_domain))
print("total_num_non_domain: "+str(total_num_non_domain))

lm_non_domain_path = tmp_dir+"lm_"+language_model+"_non_domain"
train_non_domain_tmp = tmp_dir+"train_non_domain.txt"

if not os.path.exists(train_non_domain_tmp):
    shutil.copyfile(train_non_domain, train_non_domain_tmp)

### word clustering
print("cd "+tmp_dir+"; "+\
      wcluster+\
      " --text "+train_non_domain_tmp+\
      " --c "+str(cluster_size))

### adapt the output of wcluster to something that theanoLM recognizes as vocab
vocab_path = tmp_dir+"train_non_domain-c"+str(cluster_size)+"-p1.out/paths"
if not os.path.exists(vocab_path+"3"):
    sh("cut -f1,2 "+vocab_path+" > "+vocab_path+"2")
    sh("awk -F $'\t' ' { t = $1; $1 = $2; $2 = t; print; } ' OFS=$'\t' "+\
       vocab_path+"2"+\
       " > "+vocab_path+"3")

print("theanolm train "+lm_non_domain_path+\
      " --training-set "+train_non_domain+\
      " --validation-file "+dev_in_domain+\
      " --vocabulary "+vocab_path+"3"+\
      " --vocabulary-format classes "+\
      " --architecture "+net_arch+\
      " --learning-rate "+str(learning_rate)+\
      " --optimization-method "+optimization_method+\
      " --stopping-criterion no-improvement"+\
      " --validation-frequency 8"+\
      " --patience 4")

Sample file exists at: /home/ec2-user/kklab/Projects/lrlp/experiment_elisa.il3-eng.y1r1.v2/oov_trans_neural/train_non_domain
total_num_in_domain: 99005
total_num_non_domain: 99005
cd /home/ec2-user/kklab/Projects/lrlp/experiment_elisa.il3-eng.y1r1.v2/oov_trans_neural/; /home/ec2-user/kklab/src/brown-cluster/wcluster --text /home/ec2-user/kklab/Projects/lrlp/experiment_elisa.il3-eng.y1r1.v2/oov_trans_neural/train_non_domain.txt --c 50
theanolm train /home/ec2-user/kklab/Projects/lrlp/experiment_elisa.il3-eng.y1r1.v2/oov_trans_neural/lm_50_lstm1500_1.0_nesterov_non_domain --training-set /home/ec2-user/kklab/Projects/lrlp/experiment_elisa.il3-eng.y1r1.v2/oov_trans_neural/train_non_domain --validation-file /home/ec2-user/kklab/data/ELISA/evals/y1/JMIST/elisa.il3.package.y1r1.v2/dev/elisa.il3-eng.dev.y1r1.v2.true.eng --vocabulary /home/ec2-user/kklab/Projects/lrlp/experiment_elisa.il3-eng.y1r1.v2/oov_trans_neural/train_non_domain-c50-p1.out/paths3 --vocabulary-format classes  --architecture

In [34]:
def score_sent(lm_model, all_sent, prob_or_entropy):
    '''
    param:
        lm_model: path to lm file
        all_sent: path to file of all sentences to be scored
    return:
        scores: list of scores for sentences
    '''
    cmd = "theanolm score "+lm_model+" "+all_sent+" --output word-scores --log-base 10"
    stdout = !($cmd)
    probs = []
    for item in stdout:
        if 'log(p(</s> |' in item:
            probs.append(float(item.split('=')[-1]))
    print(len(probs))
#     if prob_or_entropy == "prob":
#         return probs
#     elif prob_or_entropy == "entropy":
#         cross_ent = []
#         ctr = 0
#         with open(all_sent) as f:
#             for sent in f:
#                 cross_ent.append(-probs[ctr]*1.0/len(sent.split(' ')))
#                 ctr += 1
#         return cross_ent
#     else:
#         return -1

### heap
denominator = 2.0
total_num_non_domain_all = get_file_length(train_non_domain_all)
cutoff_num_non_domain = total_num_non_domain_all/denominator
print("scoring every sentence in non-domain training using in-domain lm...")
#cross_entropy_in_domain = 
score_sent(lm_in_domain_path, \
                                     train_non_domain_all, \
                                     "entropy")
# print("scoring every sentence in non-domain training using non-domain lm...")
# cross_entropy_non_domain = score_sent(lm_non_domain_path, \
#                                       train_non_domain_all, \
#                                       "entropy")
# cross_entropy_diff = [-(cross_entropy_in_domain[i]-cross_entropy_non_domain[i]) for i in range(len(cross_entropy_in_domain))]
# print("difference in cross entropy is obtained!")

scoring every sentence in non-domain training using in-domain lm...
^C
0


In [ ]:
score_sent = []
with open(train_non_domain_all) as f:
    for i in range(len(all_sent)):
        if len(score_sent) < cutoff_num_non_domain:
            heapq.heappush(score_sent, (cross_entropy_diff[i], all_sent[i]))
        else:
            spilled = heapq.heappushpop(score_sent, (cross_entropy_diff[i], all_sent[i]))
print("1/"+str(denominator)+" of the of non-domain training data has been loaded to heap.")
print("----------------")

In [22]:
a = !"ls"
print(a)

['elisa.il3-eng.train.y1r1.v2.true-c50-p1.out', 'elisa.il3-eng.train.y1r1.v2.true.eng', 'oov_trans_neural.ipynb']


In [4]:
# -------- hyperparameters specific to this method --------
### ug_dict or eng_vocab
candidate_source = "ug_dict"
### True or False, only meaningful when candidate_source == ug_dict
add_aligned_oov = True
# ### neural
# language_model = "neural"

# # -------- write --------
# res_file = ".".join([tra_file,\
#                      "oovtranslated",\
#                      candidate_source,\
#                      "lm_"+language_model])
# if candidate_source is "ug_dict":
#     if add_aligned_oov:
#         res_file = ".".join([tra_file,\
#                              "oovtranslated",\
#                              candidate_source+"_withAlignedOov",\
#                              "lm_"+language_model])
#     else:
#         res_file = ".".join([tra_file,\
#                              "oovtranslated",\
#                              candidate_source+"_withoutAlignedOov",\
#                              "lm_"+language_model])

# -------- translate --------
#oov_trans_neural(candidate_source, add_aligned_oov, language_model, res_file)